In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [1]:
from pathlib import Path

import pandas as pd
import seaborn as sns
from artifact_core.libs.resource_spec.binary.spec import BinaryFeatureSpec
from artifact_experiment.libs.tracking.filesystem.client import FilesystemTrackingClient
from matplotlib import pyplot as plt

from demos.binary_classification.config.constants import (
    LABEL_FEATURE,
    LS_CATEGORIES,
    LS_FEATURES,
    POSITIVE_CATEGORY,
    TRAINING_DATASET_PATH,
)
from demos.binary_classification.mlp_classifier import MLPClassifier

KeyError: 'ls_features'

In [4]:
Path().absolute()

WindowsPath('c:/Users/hecto/Codebase/artifact-ml/artifact-torch/demos/binary_classification')

In [ ]:
sns.set_theme(style="whitegrid", palette="colorblind")

In [ ]:
artifact_experiment_root = Path().absolute().parent.parent

df_train = pd.read_csv(artifact_experiment_root / TRAINING_DATASET_PATH)

df_train

In [ ]:
class_spec = BinaryFeatureSpec(
    ls_categories=LS_CATEGORIES, positive_category=POSITIVE_CATEGORY, feature_name=LABEL_FEATURE
)

In [ ]:
filesystem_tracker = FilesystemTrackingClient.build(experiment_id="demo")

In [ ]:
model = MLPClassifier.build(class_spec=class_spec, ls_features=LS_FEATURES)

In [ ]:
epoch_scores = model.fit(
    df=df_train, class_spec=class_spec, ls_features=LS_FEATURES, tracking_client=filesystem_tracker
)

In [ ]:
plt.figure(figsize=(10, 6))
epoch_scores["TRAIN_LOSS"].plot(color=sns.color_palette("colorblind")[2], linewidth=2)
plt.title("Train Loss", fontsize=16, fontweight="bold")
plt.xlabel("Epoch", fontsize=14)
plt.ylabel("Training Loss", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.6)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()